# Import required libraries

In [1]:
# General imports
import numpy as np
import pandas as pd
from typing import Literal, Iterable, Union
from copy import deepcopy
from tqdm import tqdm

# Sklearn model selection and metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Sklearn base estimator and ensemble models
from sklearn.base import BaseEstimator
from sklearn.ensemble import StackingClassifier, StackingRegressor, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

# Sklearn classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Genetic Algorithm functions

### Initialize population

In [3]:
def initialize_population(num_individuals, num_variables, seed):
    np.random.seed(seed)
    pop = np.random.randint(2, size=(num_individuals, num_variables))
    return pop

### Crossover

In [4]:
def crossover(pop, mode: Literal["1X", "UX"] = "UX", seed=None, crossover_prob=0.5):
    np.random.seed(seed)
    num_individuals = len(pop)
    num_parameters = len(pop[0])
    indices = np.arange(num_individuals)
    np.random.shuffle(indices)
    offspring = []

    for i in range(0, num_individuals, 2):
        idx1 = indices[i]
        idx2 = indices[i + 1]
        offspring1 = list(pop[idx1])
        offspring2 = list(pop[idx2])

        if mode == "UX":
            for idx in range(0, num_parameters):
                r = np.random.rand()
                if r < crossover_prob:
                    temp = offspring2[idx]
                    offspring2[idx] = offspring1[idx]
                    offspring1[idx] = temp
        else:
            cross_point = np.random.randint(num_parameters - 1)
            for idx in range(cross_point, num_parameters):
                temp = offspring2[idx]
                offspring2[idx] = offspring1[idx]
                offspring1[idx] = temp

        offspring.append(offspring1)
        offspring.append(offspring2)

    offspring = np.array(offspring)
    return offspring

### Mutation

In [5]:
def mutation(pop, seed=None, mutation_prob=0.5):
    np.random.seed(seed)
    num_individuals = len(pop)
    num_parameters = len(pop[0])

    mutation_mask = np.random.choice([True, False],
                                     size=(num_individuals, num_parameters),
                                     p=[mutation_prob, 1 - mutation_prob])
    return np.where(mutation_mask, 1-pop, pop)

### Selection

In [6]:
def tournament_selection_for_popop(pop, pop_fitness, tournament_size, seed=None):
    assert pop.shape[0] == pop_fitness.shape[0]

    np.random.seed(seed)
    selected_indices = []
    indices = np.arange(pop.shape[0])

    for _ in range(2):
        np.random.shuffle(indices)

        tournament_groups = np.split(indices, np.arange(tournament_size, pop.shape[0], tournament_size))

        for group in tournament_groups:
            group_fitness = np.apply_along_axis(lambda x: pop_fitness[x], arr=group, axis=0)
            selected_indices.append(group[np.argmax(group_fitness)])

    selected_indices = np.array(selected_indices)
    return selected_indices

In [7]:
def truncation_selection(pop, pop_fitness, selection_size=None):
    assert pop.shape[0] == pop_fitness.shape[0]
    if selection_size is None:
        selection_size = pop.shape[0] // 2
    selected_indices = np.argsort(pop_fitness)[-selection_size:]
    return selected_indices

### Some misc functions

In [8]:
def merge_two_population(pop1, pop2):
    result_pop = np.vstack([pop1, pop2])
    return result_pop

def pop_from_selection(pop, selected_indices):
    result_pop = np.array([pop[i] for i in selected_indices])
    return result_pop

def is_converged(pop: np.ndarray):
    return np.unique(pop).shape[0] == 1

def is_best_fitness_possible(pop: np.ndarray, best_fitness, func):
    num_params = pop[0].shape[0]
    best_ind = np.ones(num_params)
    return best_fitness == func(best_ind)

# Genetic Algorithms

## Paper-purposed algorithm

In [9]:
def paper_genetic_algorithm(fitness_function: callable,
                            num_individuals: int,
                            num_parameters: int,
                            seed=None,
                            num_generations: int=10,
                            mutation_rate: float=0.5,
                            crossover_rate: float=0.5,
                            verbose=False):
    # Step 1: Initialize Population
    population = initialize_population(num_individuals, num_parameters, seed)
    gen_counter = 0
    eval_counter = 0
    best_fitness = []
    fitness_scores = None  # Initialize fitness_scores as None

    if verbose:
        print(f"Gen #{gen_counter}:\n{population}")

    # Step 2: Evolutionary loop
    for generation in range(num_generations):
        # Step 3: Evaluation (only if fitness_scores is None)
        if fitness_scores is None:
            fitness_scores = np.array([fitness_function(individual) for individual in population])
            eval_counter += fitness_scores.shape[0]

        gen_counter += 1
        best_fitness.append([eval_counter, np.max(fitness_scores)])

        # Step 4: Selection
        selected_individuals = truncation_selection(population, fitness_scores)
        selected_population = pop_from_selection(population, selected_individuals)
        selected_fitness_scores = fitness_scores[selected_individuals]  # Extract fitness scores for selected individuals

        # Step 5: Crossover
        offspring = crossover(population, crossover_prob=crossover_rate)

        # Step 6: Mutation
        mutated_offspring = mutation(offspring, mutation_prob=mutation_rate)
        offspring_fitness_scores = np.array([fitness_function(individual) for individual in mutated_offspring])
        eval_counter += offspring_fitness_scores.shape[0]

        mutated_selection = truncation_selection(mutated_offspring, offspring_fitness_scores)
        mutated_offspring = pop_from_selection(mutated_offspring, mutated_selection)
        selected_offspring_fitness_scores = offspring_fitness_scores[mutated_selection]  # Extract fitness scores for mutated selection

        # Step 7: Create new population
        population = merge_two_population(selected_population, mutated_offspring)
        fitness_scores = np.concatenate((selected_fitness_scores, selected_offspring_fitness_scores))  # Merge fitness scores

        if verbose:
            print(f"Gen #{gen_counter}:\n{population}")

        # Step 8: Check for convergence
        if is_converged(population):
            break

    if verbose:
        print('#Final result:')
        print(population)
        print(best_fitness)

    return (population,
            fitness_scores,
            best_fitness,
            eval_counter,
            is_converged(population),
            is_best_fitness_possible(population, np.max(fitness_scores), fitness_function))

## POPOP algorithm

In [10]:
def popop_genetic_algorithm(metric, num_individuals, num_parameters,
                            crossover_mode: Literal["1X", "UX"] = "UX",
                            max_evaluations=100_000, seed=None, verbose=False,
                            crossover_prob=0.5, mutation_prob=0.1):
    pop = initialize_population(num_individuals, num_parameters, seed)
    np.random.seed(seed)

    num_eval = 0
    generation_num = 0
    best_fitness = []

    if verbose:
        print(f"Gen #0:\n{pop}")

    while num_eval < max_evaluations:
        offspring = crossover(pop, crossover_mode, seed, crossover_prob=crossover_prob)
        offspring = mutation(offspring, seed, mutation_prob=mutation_prob)
        pop = merge_two_population(pop, offspring)

        pop_fitness = np.array([metric(ind) for ind in pop])
        num_eval += pop_fitness.shape[0]
        best_fitness.append([num_eval, np.max(pop_fitness)])

        selection_indices = tournament_selection_for_popop(pop, pop_fitness, 4, seed)
        pop = pop_from_selection(pop, selection_indices)
        generation_num += 1
        if verbose:
            print(f"Gen #{generation_num}:\n{pop}")

        if is_converged(pop):
            break

    pop_fitness = np.array([metric(ind) for ind in pop])
    best_fitness.append([num_eval, np.max(pop_fitness)])
    if verbose:
        print('#Final result:')
        print(pop)
        print(pop_fitness)

    return (pop,
            pop_fitness,
            best_fitness,
            num_eval,
            is_converged(pop),
            is_best_fitness_possible(pop, np.max(pop_fitness), metric))

# Tuning models

In [11]:
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from copy import deepcopy

def tune_and_evaluate_models(X_train, X_val, y_train, y_val, base_models, tuning_params, metric, mode='auto', n_iter=20, cv=5, seed=None):
    """
    Tune and evaluate models using GridSearchCV or RandomizedSearchCV based on the specified mode.

    Parameters:
    - X_train, X_val: Training and validation features
    - y_train, y_val: Training and validation targets
    - base_models: Dictionary of models to evaluate
    - tuning_params: Dictionary of hyperparameter grids for tuning
    - metric: Evaluation metric
    - mode: 'exhaustive', 'grid', 'randomized', or 'auto'
    - n_iter: Number of iterations for RandomizedSearchCV, or number of max iterations for 'auto'
    - cv: Cross-validation strategy or number of folds

    Returns:
    - optimal_models: Dictionary of models fitted with the best parameters
    - validation_performance: Dictionary of performance metrics and best parameters
    """
    # Create a scorer from the metric (e.g., f1_score)
    scorer = make_scorer(metric)

    # Dictionary to store models with optimal parameters
    optimal_models = {}
    # Dictionary to store model performance on validation set
    validation_performance = {}

    # Iterate through each model in the base_models dictionary
    for model_name, model in tqdm(base_models.items(), desc="Tuning Models", unit="model"):
        # Make a deepcopy of the model to avoid altering the original
        model_copy = deepcopy(model)

        # Check if the model has tuning parameters provided in tuning_params
        if model_name in tuning_params:
            param_grid = tuning_params[model_name]

            # Determine the total number of parameter combinations
            num_combinations = np.prod([len(values) for values in param_grid.values()])

            # Decide search method based on mode
            if mode in ['exhaustive', 'grid']:
                search_method = GridSearchCV
                search_kwargs = {'param_grid': param_grid}
            elif mode == 'randomized':
                search_method = RandomizedSearchCV
                search_kwargs = {'param_distributions': param_grid, 'n_iter': n_iter}
            elif mode == 'auto':
                if num_combinations <= n_iter:
                    search_method = GridSearchCV
                    search_kwargs = {'param_grid': param_grid}
                else:
                    search_method = RandomizedSearchCV
                    search_kwargs = {'param_distributions': param_grid, 'n_iter': min(n_iter, num_combinations), 'random_state': seed}
            else:
                raise ValueError(f"Unknown mode '{mode}'. Choose from 'exhaustive', 'grid', 'randomized', 'auto'.")

            # Initialize the chosen search method with cross-validation
            grid_or_random_search = search_method(
                estimator=model_copy,
                scoring=scorer,
                cv=cv,
                n_jobs=-1,
                **search_kwargs
            )

            # Fit the model to the training data
            grid_or_random_search.fit(X_train, y_train)

            # Extract the best model and its parameters
            best_params = grid_or_random_search.best_params_
            best_score = grid_or_random_search.best_score_

            # Equip the original model with the best parameters
            best_model = clone(model)
            best_model.set_params(**best_params)

            # Fit the model with the optimal parameters
            best_model.fit(X_train, y_train)

            # Store the fitted model
            optimal_models[model_name] = best_model

            # Evaluate model performance on the validation set
            val_score = metric(y_val, best_model.predict(X_val))
            validation_performance[model_name] = {
                "best_params": best_params,
                "cv_best_score": best_score,
                "val_score": val_score
            }
        else:
            # If no tuning params are provided, use the model as is
            model_copy.fit(X_train, y_train)

            # Store the fitted model
            optimal_models[model_name] = model_copy

            # Evaluate on validation set without tuning
            val_score = metric(y_val, model_copy.predict(X_val))
            validation_performance[model_name] = {
                "best_params": None,
                "cv_best_score": None,
                "val_score": val_score
            }

    return optimal_models, validation_performance

# GA-stacking

In [12]:
class GAStackingSolver:
    def __init__(self, X_train, y_train, X_val, y_val,
                 base_models: Union[list[BaseEstimator], dict[str, BaseEstimator]],
                 meta_model: BaseEstimator,
                 num_individuals, metric,
                 metric_mode: Literal['error', 'accuracy'] = 'accuracy',
                 task: Literal['regressor', 'classifier'] = 'classifier',
                 ga_mode: Literal['paper', 'popop'] = 'paper',
                 max_gen=10,
                 crossover_prob=0.2,
                 mutation_prob=0.2,
                 verbose=False,
                 seed=None,
                 **kwargs):
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.base_models = base_models
        self.meta_model = meta_model
        self.num_individuals = num_individuals
        self.metric = metric
        self.metric_mode = metric_mode
        self.task = task
        self.ga_mode = ga_mode
        self.max_gen = max_gen
        self.crossover_prob = crossover_prob
        self.mutation_prob = mutation_prob
        self.verbose = verbose
        self.seed = seed
        self.kwargs = kwargs

        self.best_model = None
        self.best_fitness = -np.inf if metric_mode == 'accuracy' else np.inf
        self.best_combination = None

    def _fitness(self, ind: np.ndarray):
        num_params = ind.shape[0]

        # Prepare base models based on the individual's chromosome
        if isinstance(self.base_models, dict):
            base_models_copy = [
                (name, deepcopy(model)) for i, (name, model) in enumerate(self.base_models.items()) if ind[i]
            ]
        else:  # self.base_models is assumed to be a list
            base_models_copy = [
                (f"estimator_{i}", deepcopy(self.base_models[i])) for i in range(num_params) if ind[i]
            ]

        # Ensure we don't end up with an empty base model list
        if not base_models_copy:
            return -np.inf if self.metric_mode == 'accuracy' else np.inf

        meta_model_copy = deepcopy(self.meta_model)
        if self.task == 'classifier':
            model = StackingClassifier(estimators=base_models_copy, final_estimator=meta_model_copy, cv='prefit', **self.kwargs)
        else:
            model = StackingRegressor(estimators=base_models_copy, final_estimator=meta_model_copy, cv='prefit', **self.kwargs)

        model.fit(self.X_train, self.y_train)
        y_val_predict = model.predict(self.X_val)
        fitness_value = self.metric(self.y_val, y_val_predict)

        if self.metric_mode == 'accuracy':
            if fitness_value > self.best_fitness:
                self.best_fitness = fitness_value
                self.best_model = model
                self.best_combination = np.copy(ind)
        else:
            if fitness_value < self.best_fitness:
                self.best_fitness = fitness_value
                self.best_model = model
                self.best_combination = np.copy(ind)

        return fitness_value if self.metric_mode == 'accuracy' else -fitness_value

    def solve(self):
        if self.ga_mode == 'popop':
            return popop_genetic_algorithm(
                self._fitness, self.num_individuals, len(self.base_models),
                max_evaluations=self.max_gen * 2 * self.num_individuals,
                crossover_prob=self.crossover_prob,
                mutation_prob=self.mutation_prob,
                verbose=self.verbose,
                seed=self.seed
            )
        else:
            return paper_genetic_algorithm(
                self._fitness, self.num_individuals, len(self.base_models),
                num_generations=self.max_gen,
                crossover_rate=self.crossover_prob,
                mutation_rate=self.mutation_prob,
                verbose=self.verbose,
                seed=self.seed
            )

def ga_stacking(X_train, y_train, X_val, y_val,
                base_models: Union[list[BaseEstimator], dict[str, BaseEstimator]],
                meta_model: BaseEstimator,
                num_individuals, metric,
                metric_mode: Literal['error', 'accuracy'] = 'accuracy',
                task: Literal['regressor', 'classifier'] = 'classifier',
                ga_mode: Literal['paper', 'popop'] = 'paper',
                max_gen=10,
                crossover_prob=0.2,
                mutation_prob=0.2,
                verbose=False,
                seed=None,
                **kwargs):
    return GAStackingSolver(X_train, y_train, X_val, y_val,
                            base_models,
                            meta_model,
                            num_individuals, metric,
                            metric_mode,
                            task,
                            ga_mode,
                            max_gen,
                            crossover_prob,
                            mutation_prob,
                            verbose,
                            seed,
                            **kwargs)

# Experiment

## Install AI4I 2020 PdM dataset

In [13]:
!pip install ucimlrepo

## Fetch dataset

In [14]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
ai4i_2020_predictive_maintenance_dataset = fetch_ucirepo(id=601)

# data (as pandas dataframes)
X = ai4i_2020_predictive_maintenance_dataset.data.features
y = ai4i_2020_predictive_maintenance_dataset.data.targets

## Get metadata

In [15]:
# metadata
ai4i_2020_predictive_maintenance_dataset.metadata

{'uci_id': 601,
 'name': 'AI4I 2020 Predictive Maintenance Dataset',
 'repository_url': 'https://archive.ics.uci.edu/dataset/601/ai4i+2020+predictive+maintenance+dataset',
 'data_url': 'https://archive.ics.uci.edu/static/public/601/data.csv',
 'abstract': 'The AI4I 2020 Predictive Maintenance Dataset is a synthetic dataset that reflects real predictive maintenance data encountered in industry.',
 'area': 'Computer Science',
 'tasks': ['Classification', 'Regression', 'Causal-Discovery'],
 'characteristics': ['Multivariate', 'Time-Series'],
 'num_instances': 10000,
 'num_features': 6,
 'feature_types': ['Real'],
 'demographics': [],
 'target_col': ['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'],
 'index_col': ['UID', 'Product ID'],
 'has_missing_values': 'no',
 'missing_values_symbol': None,
 'year_of_dataset_creation': 2020,
 'last_updated': 'Wed Feb 14 2024',
 'dataset_doi': '10.24432/C5HS5C',
 'creators': [],
 'intro_paper': {'ID': 386,
  'type': 'NATIVE',
  'title': 'Explainab

In [16]:
# variable information
ai4i_2020_predictive_maintenance_dataset.variables

,name,role,type,demographic,description,units,missing_values
0,UID,ID,Integer,None,None,None,no
1,Product ID,ID,Categorical,None,None,None,no
2,Type,Feature,Categorical,None,None,None,no
3,Air temperature,Feature,Continuous,None,None,K,no
4,Process temperature,Feature,Continuous,None,None,K,no
5,Rotational speed,Feature,Integer,None,None,rpm,no
6,Torque,Feature,Continuous,None,None,Nm,no
7,Tool wear,Feature,Integer,None,None,min,no
8,Machine failure,Target,Integer,None,None,None,no
9,TWF,Target,Integer,None,None,None,no


## Get the target columns

In [17]:
y = y['Machine failure']

In [18]:
y.value_counts()

Machine failure
0    9661
1     339
Name: count, dtype: int64

### Preprocess features

In [19]:
X.columns

Index(['Type', 'Air temperature', 'Process temperature', 'Rotational speed',
       'Torque', 'Tool wear'],
      dtype='object')

In [20]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Type                 10000 non-null  object 
 1   Air temperature      10000 non-null  float64
 2   Process temperature  10000 non-null  float64
 3   Rotational speed     10000 non-null  int64  
 4   Torque               10000 non-null  float64
 5   Tool wear            10000 non-null  int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 468.9+ KB


In [21]:
X['Type'].unique()

array(['M', 'L', 'H'], dtype=object)

In [22]:
# Convert categorical to numerical features
pd.set_option('future.no_silent_downcasting', True)
X['Type'].replace(to_replace=[i for i in X['Type'].unique()],
                  value=[i for i in range(3)], inplace=True)

## Modeling

In [23]:
models = [
    ('Logistic Regression', LogisticRegression()),  # No random_state
    ('K-Nearest Neighbors', KNeighborsClassifier()),  # No random_state
    ('Decision Tree', DecisionTreeClassifier(random_state=42)),  # Added random_state
    ('Random Forest', RandomForestClassifier(random_state=42)),  # Added random_state
    ('AdaBoost', AdaBoostClassifier(random_state=42)),  # Added random_state
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42)),  # Added random_state
    ('Naive Bayes', GaussianNB()),  # No random_state
    ('Neural Network', MLPClassifier(random_state=42))  # Added random_state
]

In [24]:
tuning_params = {
    'Logistic Regression': {
        'penalty': ['l1', 'l2', 'elasticnet', 'none'],
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['lbfgs', 'liblinear', 'saga']
    },
    'K-Nearest Neighbors': {
        'n_neighbors': [3, 5, 7, 9, 11],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan', 'minkowski']
    },
    'Decision Tree': {
        'criterion': ['gini', 'entropy', 'log_loss'],
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'Random Forest': {
        'n_estimators': [50, 100, 200, 300],
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    },
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1, 10],
        'algorithm': ['SAMME', 'SAMME.R']
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 10]
    },
    'Naive Bayes': {
        'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
    },
    'Neural Network': {
        'hidden_layer_sizes': [(50,), (100,), (50, 50)],
        'activation': ['relu', 'tanh', 'logistic'],
        'solver': ['adam', 'sgd', 'lbfgs'],
        'alpha': [0.0001, 0.001, 0.01]
    }
}

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, train_size=0.8, test_size=0.1, random_state=42)

## GA-stacking with Default Parameters

In [26]:
tuned_models, _ = tune_and_evaluate_models(X_train, X_val, y_train, y_val, dict(models), {}, f1_score, mode='grid')

Tuning Models: 100%|██████████| 8/8 [00:03<00:00,  2.08model/s]


In [27]:
meta_models = [
    ('Logistic Regression', LogisticRegression()),  # No random_state
    ('K-Nearest Neighbors', KNeighborsClassifier()),  # No random_state
    ('Decision Tree', DecisionTreeClassifier(random_state=42)),  # Added random_state
    ('Random Forest', RandomForestClassifier(random_state=42)),  # Added random_state
    ('AdaBoost', AdaBoostClassifier(random_state=42)),  # Added random_state
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42)),  # Added random_state
    ('Naive Bayes', GaussianNB()),  # No random_state
    ('Neural Network', MLPClassifier(random_state=42))  # Added random_state
]

results = []

for name, meta_model in tqdm(meta_models):
    print(f"Running GA-stacking with meta model: {name}\n{'='*40}")
    solver = ga_stacking(
        X_train, y_train, X_val, y_val, tuned_models,
        meta_model=meta_model,
        num_individuals=10,
        max_gen=20,
        metric=f1_score,
        seed=42,
        ga_mode='paper'
    )
    solver.solve()
    
    best_combination = solver.best_combination
    model = solver.best_model
    y_pred = model.predict(X_test)
    precision = precision_score(y_pred=y_pred, y_true=y_test)
    recall = recall_score(y_pred=y_pred, y_true=y_test)
    f30 = (1 + 30 ** 2) * (precision * recall) / ((30 ** 2 * precision) + recall)
    f1 = f1_score(y_pred=y_pred, y_true=y_test)

    results.append({
        "Meta Model": name,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "F30 Score": f30,
        "Best Combination": best_combination
    })

# Print summary of results
def print_summary():
    print(f"{'='*40}\nSummary of Results\n{'='*40}")
    for result in results:
        print(f"Meta Model: {result['Meta Model']}\n"
              f"  • Precision: {result['Precision']}\n"
              f"  • Recall: {result['Recall']}\n"
              f"  • F1 Score: {result['F1 Score']}\n"
              f"  • F30 Score: {result['F30 Score']}\n"
              f"  • Best Combination: {result['Best Combination']}\n")

print_summary()

  0%|          | 0/8 [00:00<?, ?it/s]

Running GA-stacking with meta model: Logistic Regression


 12%|█▎        | 1/8 [01:02<07:17, 62.56s/it]

Running GA-stacking with meta model: K-Nearest Neighbors


 25%|██▌       | 2/8 [02:09<06:29, 64.85s/it]

...

Running GA-stacking with meta model: Naive Bayes


 88%|████████▊ | 7/8 [10:43<01:37, 97.02s/it] 

Running GA-stacking with meta model: Neural Network


100%|██████████| 8/8 [21:14<00:00, 159.36s/it]

Summary of Results
Meta Model: Logistic Regression
  • Precision: 0.7333333333333333
  • Recall: 0.6470588235294118
  • F1 Score: 0.6875
  • F30 Score: 0.647143323539014
  • Best Combination: [1 0 0 0 0 1 0 0]

Meta Model: K-Nearest Neighbors
  • Precision: 0.7058823529411765
  • Recall: 0.7058823529411765
  • F1 Score: 0.7058823529411765
  • F30 Score: 0.7058823529411764
  • Best Combination: [0 0 0 0 1 1 0 1]

Meta Model: Decision Tree
  • Precision: 0.6190476190476191
  • Recall: 0.7647058823529411
  • F1 Score: 0.6842105263157895
  • F30 Score: 0.7645062332745904
  • Best Combination: [1 0 0 0 1 1 1 0]

Meta Model: Random Forest
  • Precision: 0.7352941176470589
  • Recall: 0.7352941176470589
  • F1 Score: 0.735294117647059
  • F30 Score: 0.7352941176470589
  • Best Combination: [0 0 0 0 1 1 1 0]

Meta Model: AdaBoost
  • Precision: 0.7666666666666667
  • Recall: 0.6764705882352942
  • F1 Score: 0.71875
  • F30 Score: 0.6765589291544238
  • Best Combination: [0 0 0 0 1 1 1 1]

Meta

In [28]:
for tuned_model_name, tuned_model in tuned_models.items():
    y_pred = tuned_model.predict(X_test)
    
    # Print model name and its chosen parameters
    print(f'{"="*40}\nModel: {tuned_model_name}\n')
    
    # Print metrics with bullet points
    print(f'Metrics:\n'
          f'  • Precision: {precision_score(y_pred=y_pred, y_true=y_test)}\n'
          f'  • Recall: {recall_score(y_pred=y_pred, y_true=y_test)}\n'
          f'  • F1 Score: {f1_score(y_pred=y_pred, y_true=y_test)}')
    
    # Separator between models
    print("="*40)

Model: Logistic Regression

Metrics:
  • Precision: 0.7
  • Recall: 0.20588235294117646
  • F1 Score: 0.3181818181818182
Model: K-Nearest Neighbors

Metrics:
  • Precision: 0.5714285714285714
  • Recall: 0.11764705882352941
  • F1 Score: 0.1951219512195122
Model: Decision Tree

Metrics:
  • Precision: 0.631578947368421
  • Recall: 0.7058823529411765
  • F1 Score: 0.6666666666666667
Model: Random Forest

Metrics:
  • Precision: 0.8695652173913043
  • Recall: 0.5882352941176471
  • F1 Score: 0.7017543859649124
Model: AdaBoost

Metrics:
  • Precision: 0.5333333333333333
  • Recall: 0.47058823529411764
  • F1 Score: 0.5
Model: Gradient Boosting

Metrics:
  • Precision: 0.7241379310344828
  • Recall: 0.6176470588235294
  • F1 Score: 0.6666666666666667
Model: Naive Bayes

Metrics:
  • Precision: 0.3157894736842105
  • Recall: 0.17647058823529413
  • F1 Score: 0.22641509433962262
Model: Neural Network

Metrics:
  • Precision: 1.0
  • Recall: 0.08823529411764706
  • F1 Score: 0.162162162162162

## GA-stacking with Tuned Parameters

In [29]:
tuned_models, _ = tune_and_evaluate_models(X_train, X_val, y_train, y_val, dict(models), tuning_params, f1_score, mode='grid', seed=42)

Tuning Models:   0%|          | 0/8 [00:00<?, ?model/s]/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: Converge

In [30]:
meta_models = [
    ('Logistic Regression', LogisticRegression()),  # No random_state
    ('K-Nearest Neighbors', KNeighborsClassifier()),  # No random_state
    ('Decision Tree', DecisionTreeClassifier(random_state=42)),  # Added random_state
    ('Random Forest', RandomForestClassifier(random_state=42)),  # Added random_state
    ('AdaBoost', AdaBoostClassifier(random_state=42)),  # Added random_state
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42)),  # Added random_state
    ('Naive Bayes', GaussianNB()),  # No random_state
    ('Neural Network', MLPClassifier(random_state=42))  # Added random_state
]

results = []

for name, meta_model in tqdm(meta_models):
    print(f"Running GA-stacking with meta model: {name}\n{'='*40}")
    solver = ga_stacking(
        X_train, y_train, X_val, y_val, tuned_models,
        meta_model=meta_model,
        num_individuals=10,
        max_gen=20,
        metric=f1_score,
        seed=42,
        ga_mode='paper'
    )
    solver.solve()
    
    best_combination = solver.best_combination
    model = solver.best_model
    y_pred = model.predict(X_test)
    precision = precision_score(y_pred=y_pred, y_true=y_test)
    recall = recall_score(y_pred=y_pred, y_true=y_test)
    f30 = (1 + 30 ** 2) * (precision * recall) / ((30 ** 2 * precision) + recall)
    f1 = f1_score(y_pred=y_pred, y_true=y_test)

    results.append({
        "Meta Model": name,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "F30 Score": f30,
        "Best Combination": best_combination
    })

# Print summary of results
def print_summary():
    print(f"{'='*40}\nSummary of Results\n{'='*40}")
    for result in results:
        print(f"Meta Model: {result['Meta Model']}\n"
              f"  • Precision: {result['Precision']}\n"
              f"  • Recall: {result['Recall']}\n"
              f"  • F1 Score: {result['F1 Score']}\n"
              f"  • F30 Score: {result['F30 Score']}\n"
              f"  • Best Combination: {result['Best Combination']}\n")

print_summary()

  0%|          | 0/8 [00:00<?, ?it/s]

Running GA-stacking with meta model: Logistic Regression


 12%|█▎        | 1/8 [02:01<14:13, 121.89s/it]

Running GA-stacking with meta model: K-Nearest Neighbors


 25%|██▌       | 2/8 [04:31<13:49, 138.18s/it]

...

Running GA-stacking with meta model: Naive Bayes


 88%|████████▊ | 7/8 [18:05<02:38, 158.42s/it]

Running GA-stacking with meta model: Neural Network


100%|██████████| 8/8 [27:17<00:00, 204.67s/it]

Summary of Results
Meta Model: Logistic Regression
  • Precision: 0.8
  • Recall: 0.7058823529411765
  • F1 Score: 0.7500000000000001
  • F30 Score: 0.7059745347698335
  • Best Combination: [0 0 0 0 1 1 0 0]

Meta Model: K-Nearest Neighbors
  • Precision: 0.7857142857142857
  • Recall: 0.6470588235294118
  • F1 Score: 0.7096774193548386
  • F30 Score: 0.6471855818205564
  • Best Combination: [1 1 1 1 1 1 0 0]

Meta Model: Decision Tree
  • Precision: 0.7741935483870968
  • Recall: 0.7058823529411765
  • F1 Score: 0.7384615384615385
  • F30 Score: 0.7059514870555972
  • Best Combination: [0 0 1 1 1 1 1 0]

Meta Model: Random Forest
  • Precision: 0.8275862068965517
  • Recall: 0.7058823529411765
  • F1 Score: 0.7619047619047619
  • F30 Score: 0.70599758398903
  • Best Combination: [1 0 0 1 0 1 0 0]

Meta Model: AdaBoost
  • Precision: 0.7741935483870968
  • Recall: 0.7058823529411765
  • F1 Score: 0.7384615384615385
  • F30 Score: 0.7059514870555972
  • Best Combination: [0 1 0 0 0 1 0 

In [31]:
for tuned_model_name, tuned_model in tuned_models.items():
    y_pred = tuned_model.predict(X_test)
    
    # Print model name and its chosen parameters
    print(f'{"="*40}\nModel: {tuned_model_name}\n')
    
    # Extract the chosen parameters for the current model from `tuning_params`
    chosen_params = tuning_params.get(tuned_model_name, {})
    
    # Print only the chosen parameters
    if chosen_params:
        print("Chosen Parameters:")
        for param, values in chosen_params.items():
            print(f'  • {param}: {tuned_model.get_params().get(param, "Not Set")}')
    
    print()  # Newline after parameters
    
    # Print metrics with bullet points
    print(f'Metrics:\n'
          f'  • Precision: {precision_score(y_pred=y_pred, y_true=y_test)}\n'
          f'  • Recall: {recall_score(y_pred=y_pred, y_true=y_test)}\n'
          f'  • F1 Score: {f1_score(y_pred=y_pred, y_true=y_test)}')
    
    # Separator between models
    print("="*40)

Model: Logistic Regression

Chosen Parameters:
  • penalty: l1
  • C: 100
  • solver: liblinear

Metrics:
  • Precision: 0.7777777777777778
  • Recall: 0.20588235294117646
  • F1 Score: 0.3255813953488372
Model: K-Nearest Neighbors

Chosen Parameters:
  • n_neighbors: 3
  • weights: distance
  • metric: manhattan

Metrics:
  • Precision: 0.4666666666666667
  • Recall: 0.20588235294117646
  • F1 Score: 0.28571428571428564
Model: Decision Tree

Chosen Parameters:
  • criterion: gini
  • max_depth: 10
  • min_samples_split: 2
  • min_samples_leaf: 1

Metrics:
  • Precision: 0.6857142857142857
  • Recall: 0.7058823529411765
  • F1 Score: 0.6956521739130436
Model: Random Forest

Chosen Parameters:
  • n_estimators: 300
  • criterion: gini
  • max_depth: None
  • min_samples_split: 5

Metrics:
  • Precision: 0.8
  • Recall: 0.5882352941176471
  • F1 Score: 0.6779661016949153
Model: AdaBoost

Chosen Parameters:
  • n_estimators: 200
  • learning_rate: 1
  • algorithm: SAMME

Metrics:
  • Prec

## GA-stacking with Tuned Parameters and SMOTE technique

In [32]:
!pip install imbalanced-learn

In [33]:
from imblearn.over_sampling import SMOTE

# Calculate the class distribution
class_counts = y_train.value_counts()

# Identify the majority and minority class sizes
majority_class_size = class_counts.max()
minority_class_size = class_counts.min()

# Calculate the desired minority class size (20% of majority class size)
desired_minority_class_size = int(0.2 * majority_class_size)

# Calculate the number of new samples required for the minority class
sampling_strategy = desired_minority_class_size / majority_class_size

# Create and apply SMOTE with the custom sampling strategy
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)

X_train_sampled, y_train_sampled = smote.fit_resample(X_train, y_train)

In [34]:
tuned_models, _ = tune_and_evaluate_models(X_train_sampled, X_val, y_train_sampled, y_val, dict(models), {}, f1_score, mode='grid')

Tuning Models: 100%|██████████| 8/8 [00:04<00:00,  1.66model/s]


In [35]:
meta_models = [
    ('Logistic Regression', LogisticRegression()),  # No random_state
    ('K-Nearest Neighbors', KNeighborsClassifier()),  # No random_state
    ('Decision Tree', DecisionTreeClassifier(random_state=42)),  # Added random_state
    ('Random Forest', RandomForestClassifier(random_state=42)),  # Added random_state
    ('AdaBoost', AdaBoostClassifier(random_state=42)),  # Added random_state
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42)),  # Added random_state
    ('Naive Bayes', GaussianNB()),  # No random_state
    ('Neural Network', MLPClassifier(random_state=42))  # Added random_state
]

results = []

for name, meta_model in tqdm(meta_models):
    print(f"Running GA-stacking with meta model: {name}\n{'='*40}")
    solver = ga_stacking(
        X_train_sampled, y_train_sampled, X_val, y_val, tuned_models,
        meta_model=meta_model,
        num_individuals=10,
        max_gen=20,
        metric=f1_score,
        seed=42,
        ga_mode='paper'
    )
    solver.solve()
    
    best_combination = solver.best_combination
    model = solver.best_model
    y_pred = model.predict(X_test)
    precision = precision_score(y_pred=y_pred, y_true=y_test)
    recall = recall_score(y_pred=y_pred, y_true=y_test)
    f30 = (1 + 30 ** 2) * (precision * recall) / ((30 ** 2 * precision) + recall)
    f1 = f1_score(y_pred=y_pred, y_true=y_test)

    results.append({
        "Meta Model": name,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "F30 Score": f30,
        "Best Combination": best_combination
    })

# Print summary of results
def print_summary():
    print(f"{'='*40}\nSummary of Results\n{'='*40}")
    for result in results:
        print(f"Meta Model: {result['Meta Model']}\n"
              f"  • Precision: {result['Precision']}\n"
              f"  • Recall: {result['Recall']}\n"
              f"  • F1 Score: {result['F1 Score']}\n"
              f"  • F30 Score: {result['F30 Score']}\n"
              f"  • Best Combination: {result['Best Combination']}\n")

print_summary()

  0%|          | 0/8 [00:00<?, ?it/s]

Running GA-stacking with meta model: Logistic Regression


 12%|█▎        | 1/8 [01:21<09:29, 81.29s/it]

Running GA-stacking with meta model: K-Nearest Neighbors


 25%|██▌       | 2/8 [02:46<08:23, 83.87s/it]

...

Running GA-stacking with meta model: Naive Bayes


 88%|████████▊ | 7/8 [12:08<01:45, 105.13s/it]

Running GA-stacking with meta model: Neural Network


100%|██████████| 8/8 [23:36<00:00, 177.06s/it]

Summary of Results
Meta Model: Logistic Regression
  • Precision: 0.6216216216216216
  • Recall: 0.6764705882352942
  • F1 Score: 0.6478873239436619
  • F30 Score: 0.6764043476841728
  • Best Combination: [1 0 0 1 1 0 0 1]

Meta Model: K-Nearest Neighbors
  • Precision: 0.5789473684210527
  • Recall: 0.6470588235294118
  • F1 Score: 0.6111111111111113
  • F30 Score: 0.6469743455839155
  • Best Combination: [1 0 0 1 1 1 1 1]

Meta Model: Decision Tree
  • Precision: 0.6111111111111112
  • Recall: 0.6470588235294118
  • F1 Score: 0.6285714285714287
  • F30 Score: 0.6470165817991906
  • Best Combination: [1 1 1 1 1 1 0 0]

Meta Model: Random Forest
  • Precision: 0.5945945945945946
  • Recall: 0.6470588235294118
  • F1 Score: 0.619718309859155
  • F30 Score: 0.6469954630022522
  • Best Combination: [1 1 0 1 0 0 1 0]

Meta Model: AdaBoost
  • Precision: 0.6111111111111112
  • Recall: 0.6470588235294118
  • F1 Score: 0.6285714285714287
  • F30 Score: 0.6470165817991906
  • Best Combination:

In [36]:
for tuned_model_name, tuned_model in tuned_models.items():
    y_pred = tuned_model.predict(X_test)
    
    # Print model name and its chosen parameters
    print(f'{"="*40}\nModel: {tuned_model_name}\n')
    
    # Extract the chosen parameters for the current model from `tuning_params`
    chosen_params = tuning_params.get(tuned_model_name, {})
    
    # Print only the chosen parameters
    if chosen_params:
        print("Chosen Parameters:")
        for param, values in chosen_params.items():
            print(f'  • {param}: {tuned_model.get_params().get(param, "Not Set")}')
    
    print()  # Newline after parameters
    
    # Print metrics with bullet points
    print(f'Metrics:\n'
          f'  • Precision: {precision_score(y_pred=y_pred, y_true=y_test)}\n'
          f'  • Recall: {recall_score(y_pred=y_pred, y_true=y_test)}\n'
          f'  • F1 Score: {f1_score(y_pred=y_pred, y_true=y_test)}')
    
    # Separator between models
    print("="*40)

Model: Logistic Regression

Chosen Parameters:
  • penalty: l2
  • C: 1.0
  • solver: lbfgs

Metrics:
  • Precision: 0.3548387096774194
  • Recall: 0.6470588235294118
  • F1 Score: 0.4583333333333333
Model: K-Nearest Neighbors

Chosen Parameters:
  • n_neighbors: 5
  • weights: uniform
  • metric: minkowski

Metrics:
  • Precision: 0.23943661971830985
  • Recall: 0.5
  • F1 Score: 0.32380952380952377
Model: Decision Tree

Chosen Parameters:
  • criterion: gini
  • max_depth: None
  • min_samples_split: 2
  • min_samples_leaf: 1

Metrics:
  • Precision: 0.39655172413793105
  • Recall: 0.6764705882352942
  • F1 Score: 0.5
Model: Random Forest

Chosen Parameters:
  • n_estimators: 100
  • criterion: gini
  • max_depth: None
  • min_samples_split: 2

Metrics:
  • Precision: 0.5945945945945946
  • Recall: 0.6470588235294118
  • F1 Score: 0.619718309859155
Model: AdaBoost

Chosen Parameters:
  • n_estimators: 50
  • learning_rate: 1.0
  • algorithm: SAMME.R

Metrics:
  • Precision: 0.3636363